### Import bank data from csv

In [ ]:
import pandasai as pai
import pandas as pd
import ast
from dotenv import load_dotenv
import os 

load_dotenv()

pai_api_key = os.getenv("OPENAI_API_KEY")
if not pai_api_key:
    raise ValueError("PAI_API_KEY is not set")
else:
    print(f"OPENAI_API_KEY is set")

df = pd.read_csv("bank_data_23-25/barclays_072.csv")

# Sample DataFrame
df = pai.DataFrame(df)

pai.api_key.set(pai_api_key)


### extract currency and amount
df['currency'] = df['transactionAmount'].apply(lambda x: ast.literal_eval(x)['currency'])
df['amount'] = df['transactionAmount'].apply(lambda x: float(ast.literal_eval(x)['amount']))
df['amount'] = (df['amount'] * 100).astype(int)
df = df.drop('transactionAmount', axis=1)

## setting datetimeindex
df['bookingDate'] = pd.to_datetime(df['bookingDate'])
df.set_index('bookingDate', inplace=True)
df = df.drop(['valueDate', 'bookingDateTime', 'valueDateTime', 'internalTransactionId'], axis=1)
df = df.rename(columns={'remittanceInformationUnstructured': 'remittanceInfo'})

# df.chat("What are the total expenses for 2024?")


OPENAI_API_KEY is set


### XERO TIME!!

In [5]:
BankTransactions = { 
    "BankTransactions": [ { "Type": "SPEND", 
                           "Contact": { "ContactID": "00000220-0000-0000-0000-000000000000" }, 
                            "Lineitems": [ { "Description": "Foobar", "Quantity": 1, "UnitAmount": 20, "AccountCode": "400" } ], 
                            "BankAccount": { "Code": "088" } 
                            } ] 
}     


""" PUT https://api.xero.com/api.xro/2.0/BankTransactions
    EXAMPLE BODY GET REQUEST
{
  "BankTransactions": [
    {
      "Type": "SPEND",
      "Contact": {
        "ContactID": "ea791a0a-081c-4833-a4f1-3cccb323ec4a"  
      },
      "LineItems": [
        {
          "Description": "Foobar",
          "Quantity": 1.0,
          "UnitAmount": 20.0,
          "AccountCode": "433" 
        }
      ],
      "BankAccount": {
        "Code": "600" 
      }
    }
  ]
}
"""

' PUT https://api.xero.com/api.xro/2.0/BankTransactions\n    EXAMPLE BODY GET REQUEST\n{\n  "BankTransactions": [\n    {\n      "Type": "SPEND",\n      "Contact": {\n        "ContactID": "ea791a0a-081c-4833-a4f1-3cccb323ec4a"  \n      },\n      "LineItems": [\n        {\n          "Description": "Foobar",\n          "Quantity": 1.0,\n          "UnitAmount": 20.0,\n          "AccountCode": "433" \n        }\n      ],\n      "BankAccount": {\n        "Code": "600" \n      }\n    }\n  ]\n}\n'

### LLM Reconciliation Time

- Add new columns "coa_agent", "coa_reason", "coa_agent_confidence" for LLM


In [9]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
import time
from typing import Dict, Tuple
import logging
import json


# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

load_dotenv()

# Initialize the ChatGroq model
groq_model = ChatGroq(model_name="Llama-3.3-70b-Specdec")
openai_model = ChatOpenAI(model="gpt-4o")


class TransactionClassifier:
    def __init__(self):
        logger.info("Initializing TransactionClassifier")
        self.system_prompt = """
        You are a financial expert responsible for classifying transactions into appropriate chart of accounts.
        For each transaction, you must provide:
        1. The most specific appropriate chart of account code
        2. A brief explanation of why this classification was chosen. If confidence score is low, explain why.
        3. A confidence score between 0 and 1 (e.g., 0.95 for high confidence)

        You must respond with valid JSON in the following format only:
        {
            "code": "string",
            "reasoning": "string",
            "confidence": float
        }

        You will be given the chart of accounts, you must match the transaction against those accounts. 
        You must use the code field from the chart of accounts as the code field in the response.
        """
        self.last_request_time = 0
        self.rate_limit_delay = 2  # 2 seconds between requests (30 requests/minute)

    def _rate_limit(self):
        """Implement rate limiting"""
        current_time = time.time()
        time_since_last_request = current_time - self.last_request_time
        if time_since_last_request < self.rate_limit_delay:
            delay = self.rate_limit_delay - time_since_last_request
            logger.debug(f"Rate limiting: waiting {delay:.2f} seconds")
            time.sleep(delay)
        self.last_request_time = time.time() 

    def classify_transaction(self, transaction: Dict, chart_of_accounts: list) -> Tuple[str, str, float]:
        """Classify a single transaction using the LLM"""
        logger.info(f"Processing transaction: {transaction}")
        self._rate_limit()

        # Format the transaction details for the LLM
        transaction_prompt = f"""
        Please classify the following transaction:
        Transaction: {transaction}
        
        You must match the transaction against those accounts. 
        You must use the code field from the chart of accounts. 
        Chart of accounts: {chart_of_accounts}
        """

        messages = [
            SystemMessage(content=self.system_prompt),
            HumanMessage(content=transaction_prompt)
        ]

        try:
            logger.debug("Sending request to LLM")
            # Add more visible print statements
            print("\n=== Sending request to LLM ===")
            response = openai_model.invoke(messages)
            print("\n=== Raw LLM Response ===")
            print(response.content)
            logger.debug(f"Raw LLM response: {response.content}")

            # Try to parse the response as JSON
            try:
                # First, try direct JSON parsing
                result = json.loads(response.content)
                logger.info("Successfully parsed JSON response")
            except json.JSONDecodeError:
                # If direct parsing fails, try to extract JSON from the response
                logger.warning("Direct JSON parsing failed, attempting to extract JSON from response")
                # Look for JSON-like structure in the response
                import re
                json_match = re.search(r'\{.*\}', response.content, re.DOTALL)
                if json_match:
                    result = json.loads(json_match.group())
                    logger.info("Successfully extracted and parsed JSON from response")
                else:
                    raise ValueError("No JSON structure found in response")

            # Validate the response structure
            required_keys = {'code', 'reasoning', 'confidence'}
            if not all(key in result for key in required_keys):
                missing_keys = required_keys - result.keys()
                raise ValueError(f"Missing required keys in response: {missing_keys}")

            logger.info(f"Classification successful: {result['code']}")
            return (
                result['code'],
                result['reasoning'],
                result['confidence']
            )

        except Exception as e:
            logger.error(f"Classification failed: {str(e)}", exc_info=True)
            return (
                "ERROR",
                f"Classification failed: {str(e)}",
                0.0
            )


def process_transactions(df: pd.DataFrame, chart_of_accounts: list) -> pd.DataFrame:
    """Process transactions in DataFrame and add classifications directly"""
    logger.info(f"Starting to process {len(df)} transactions")
    classifier = TransactionClassifier()

    # Create a fresh copy of the DataFrame
    df = df.copy()
    
    # Explicitly clear any previous results
    df['coa_agent'] = None
    df['coa_reason'] = None
    df['coa_confidence'] = None

    for i, (idx, row) in enumerate(df.iterrows()):
        print("row", row)
        logger.info(f"Processing transaction {idx}")
        transaction = row.to_dict()
        transaction['amount'] = transaction['amount']/100
        code, reasoning, confidence = classifier.classify_transaction(transaction, chart_of_accounts)
        
        # Update DataFrame using iloc instead of at
        df.iloc[i, df.columns.get_loc('coa_agent')] = code
        df.iloc[i, df.columns.get_loc('coa_reason')] = reasoning
        df.iloc[i, df.columns.get_loc('coa_confidence')] = confidence
        
        # Print summary of classification
        reason_preview = ' '.join(reasoning.split()[:10]) + '...' if reasoning else 'No reason provided'
        logger.info(f"Transaction {idx}:")
        logger.info(f"Account Code: {code}")
        logger.info(f"Reason Preview: {reason_preview}")
        logger.info("-" * 50)

        break

    
    # Add verification logging
    processed_count = df[df['coa_agent'].notna()].shape[0]
    logger.info(f"Number of processed transactions: {processed_count}")
    
    logger.info("Finished processing all transactions")
    return df


""" Fetch and store chart of accounts into parsed_accounts """
import json

# Read the JSON file
with open('coa.json', 'r') as file:
    data = json.load(file)

# Extract only the required fields from each account
parsed_accounts = []
for account in data['Accounts']:
    parsed_account = {
        'code': account.get('AccountID', ''),
        'name': account.get('Name', ''),
        'status': account.get('Status', ''),
        'type': account.get('Type', ''),
        'taxtype': account.get('TaxType', ''),
        'description': account.get('Description', ''),  # Note: Not present in sample but included as requested
        'class': account.get('Class', ''),
        'reportingcode': account.get('ReportingCode', '')
    }
    parsed_accounts.append(parsed_account)

transactions = []  # Create empty list to store dictionaries

for index, row in df.iterrows():
    transaction = row.to_dict()
    transactions.append(transaction)

df_processed = process_transactions(df.copy(), parsed_accounts)

df_processed

row creditorName                                                         DVLA
remittanceInformationUnstructured    DVLA-LN61PYJ  000000000056504532 DDR
currency                                                              GBP
amount                                                              -1312
coa_agent                                                            None
coa_reason                                                           None
coa_confidence                                                       None
Name: 2025-01-02 00:00:00, dtype: object

=== Sending request to LLM ===

=== Raw LLM Response ===
```json
{
    "code": "b04da64d-1580-4dd4-9de4-ac77130087d0",
    "reasoning": "The transaction is associated with DVLA, which typically indicates a vehicle-related charge, possibly for a driving license, vehicle tax, or some related fee. The 'Motor Vehicle Expenses' account best fits the nature of this expense, as it encompasses costs associated with the running of business m

,creditorName,remittanceInformationUnstructured,currency,amount,coa_agent,coa_reason,coa_confidence
bookingDate,,,,,,,
2025-01-02,DVLA,DVLA-LN61PYJ 000000000056504532 DDR,GBP,-1312,b04da64d-1580-4dd4-9de4-ac77130087d0,"The transaction is associated with DVLA, which...",0.85
2025-01-02,SUMUP *NEW LOOK BA,SUMUP *NEW LOOK BA ON 31 DEC BCC,GBP,-6000,None,None,None
2025-01-02,NaN,SAINSBURYS BANK Sainsburys Bank 31DEC 21.41 ATM,GBP,-5000,None,None,None
2024-12-27,KLARNA*ZOOM COMMUN,KLARNA*ZOOM COMMUN ON 26 DEC BCC,GBP,-1559,None,None,None
2024-12-27,THE HUB DENTAL PRA,THE HUB DENTAL PRA ON 24 DEC BCC,GBP,-5000,None,None,None
2024-12-24,AMERICAN EXP 3773 PB166643916980508,AMERICAN EXP 3773 PB166643916980508 FT,GBP,-599,None,None,None
2024-12-24,AMERICAN EXP 3773 PB313981532278988,AMERICAN EXP 3773 PB313981532278988 FT,GBP,-21320,None,None,None
2024-12-24,CRESCENT ADVISORS MICHAEL ALI DLA,CRESCENT ADVISORS MICHAEL ALI DLA FT,GBP,-30000,None,None,None
2024-12-24,Barclaycard,MR MICHAEL ALI 4929158426786006 BBP,GBP,-14538,None,None,None


In [ ]:
" NTROPY "
import requests

ntropy_api_key = os.getenv("NTROPY_API_KEY")
if not ntropy_api_key:
    raise ValueError("NTROPY_API_KEY is not set")
else:
    print(f"NTROPY_API_KEY is set")

""" CREATE NEW ACCOUNT HOLDER """
url = "https://api.ntropy.com/v3/account_holders"
headers = {
    "Accept": "application/json",
    "X-API-KEY": ntropy_api_key,
    "Content-Type": "application/json"
}

data = {
    "id": "35b927b6-6fda-40aa-93b8-95b47c2b2cad",
    "type": "business",
    "name": "Michael Ali",
    "website": "https://flowon.ai",
    "industry": "ai software"
}

response = requests.post(url, headers=headers, json=data)
print(response.json())


NTROPY_API_KEY is set
{'details': 'Account holder with provided ID (35b927b6-6fda-40aa-93b8-95b47c2b2cad) already exists'}


In [116]:
""" NTROPY BATCH PROCESS TRANSACTIONS"""
import uuid

url = "https://api.ntropy.com/v3/batches/"


data = {
        "operation": "POST /v3/transactions",
        "data": transformed_data
    }

response = requests.post(url, headers=headers, json=data)
print(response.json())


{'id': '1a2bc613-111b-49b1-b35c-77e9b1d7a2fc', 'operation': 'POST /v3/transactions', 'status': 'processing', 'created_at': '2025-02-11T18:09:18.598722+00:00', 'updated_at': '2025-02-11T18:09:18.598722+00:00', 'progress': 0, 'total': 37}


In [ ]:
""" NOW CHECK TRANSACTION STATUS """
batch_id = "1a2bc613-111b-49b1-b35c-77e9b1d7a2fc"

url = f"https://api.ntropy.com/v3/batches/{batch_id}/results"


get_batch = requests.get(url, headers=headers)

get_batch.json()

In [122]:
get_batch.json()['results'][0]



{'entities': {'counterparty': {'id': 'e393a472-b8bd-33e6-a889-3221b4eea280',
   'name': 'Driver and Vehicle Licensing Agency',
   'website': 'gov.uk.gov.uk',
   'logo': 'https://logos.ntropy.com/business_icons-government_-_tax_-_tax_payment',
   'type': 'organization'},
  'intermediaries': []},
 'categories': {'general': 'tax payment'},
 'location': {'raw_address': 'Longview Road, Swansea SA6 7JL, United Kingdom',
  'structured': {'street': 'Longview Road',
   'city': 'Swansea',
   'state': 'Swansea',
   'postcode': 'SA6 7JL',
   'country_code': 'GB',
   'country': 'United Kingdom',
   'house_number': None,
   'latitude': 51.62079,
   'longitude': -3.94323,
   'google_maps_url': 'https://www.google.com/maps/search/?api=1&query=51.62079,-3.94323',
   'apple_maps_url': 'https://maps.apple.com/?q=51.62079,-3.94323',
   'store_number': None}},
 'created_at': '2025-02-11T18:09:18.623135+00:00',
 'id': '1177539c-b570-4588-9953-d76ae4647afb'}

In [124]:
df

,creditorName,remittanceInfo,currency,amount
bookingDate,,,,
2025-01-02,DVLA,DVLA-LN61PYJ 000000000056504532 DDR,GBP,-1312
2025-01-02,SUMUP *NEW LOOK BA,SUMUP *NEW LOOK BA ON 31 DEC BCC,GBP,-6000
2025-01-02,NaN,SAINSBURYS BANK Sainsburys Bank 31DEC 21.41 ATM,GBP,-5000
2024-12-27,KLARNA*ZOOM COMMUN,KLARNA*ZOOM COMMUN ON 26 DEC BCC,GBP,-1559
2024-12-27,THE HUB DENTAL PRA,THE HUB DENTAL PRA ON 24 DEC BCC,GBP,-5000
2024-12-24,AMERICAN EXP 3773 PB166643916980508,AMERICAN EXP 3773 PB166643916980508 FT,GBP,-599
2024-12-24,AMERICAN EXP 3773 PB313981532278988,AMERICAN EXP 3773 PB313981532278988 FT,GBP,-21320
2024-12-24,CRESCENT ADVISORS MICHAEL ALI DLA,CRESCENT ADVISORS MICHAEL ALI DLA FT,GBP,-30000
2024-12-24,Barclaycard,MR MICHAEL ALI 4929158426786006 BBP,GBP,-14538


In [123]:

transaction_id = "1177539c-b570-4588-9953-d76ae4647afb"

url = f"https://api.ntropy.com/v3/transactions/{transaction_id}"

get_transaction = requests.get(url, headers=headers)

get_transaction.json()





{'id': '1177539c-b570-4588-9953-d76ae4647afb',
 'description': 'DVLA-LN61PYJ  000000000056504532 DDR',
 'date': '2025-01-02',
 'amount': 13.12,
 'entry_type': 'outgoing',
 'currency': 'GBP',
 'entities': {'counterparty': {'id': 'e393a472-b8bd-33e6-a889-3221b4eea280',
   'name': 'Driver and Vehicle Licensing Agency',
   'website': 'gov.uk.gov.uk',
   'logo': 'https://logos.ntropy.com/business_icons-government_-_tax_-_tax_payment',
   'mccs': [],
   'type': 'organization'},
  'intermediaries': []},
 'categories': {'general': 'tax payment'},
 'location': {'raw_address': 'Longview Road, Swansea SA6 7JL, United Kingdom',
  'structured': {'street': 'Longview Road',
   'city': 'Swansea',
   'state': 'Swansea',
   'postcode': 'SA6 7JL',
   'country_code': 'GB',
   'country': 'United Kingdom',
   'house_number': None,
   'latitude': 51.62079,
   'longitude': -3.94323,
   'google_maps_url': 'https://www.google.com/maps/search/?api=1&query=51.62079,-3.94323',
   'apple_maps_url': 'https://maps.ap

In [ ]:
import json 

""" DF TO JSON FIELDS"""
# Convert DataFrame to JSON
def prepare_df_for_frontend(df):
    # Reset index to make bookingDate a column
    df = df.reset_index()
    
    # Convert datetime to ISO format string
    df['bookingDate'] = df['bookingDate'].dt.strftime('%Y-%m-%dT%H:%M:%S')
    
    # Convert to JSON records format (this gives us a string)
    json_string = df.to_json(orient='records', date_format='iso')
    
    # Parse the JSON string into Python objects (list of dictionaries)
    json_data = json.loads(json_string)
    
    return json_data

json_data = prepare_df_for_frontend(df)



In [115]:
import uuid 

def transform_transaction(transaction, account_holder_id):
    # Transform a single transaction
    return {
        "id": str(uuid.uuid4()),
        "description": transaction["remittanceInfo"],
        "date": transaction["bookingDate"].split("T")[0],
        "amount": abs(transaction["amount"]/100),  # Make amount positive
        "entry_type": "outgoing" if transaction["amount"] < 0 else "incoming",
        "currency": transaction["currency"],
        "account_holder_id": account_holder_id,
        "location": {
            "country": "GB"
        }
    }

# Example usage:
account_holder_id = "35b927b6-6fda-40aa-93b8-95b47c2b2cad"
# Transform all transactions using list comprehension
transformed_data = [
    transform_transaction(transaction, account_holder_id) 
    for transaction in json_data
]

transformed_data

[{'id': '1177539c-b570-4588-9953-d76ae4647afb',
  'description': 'DVLA-LN61PYJ  000000000056504532 DDR',
  'date': '2025-01-02',
  'amount': 13.12,
  'entry_type': 'outgoing',
  'currency': 'GBP',
  'account_holder_id': '35b927b6-6fda-40aa-93b8-95b47c2b2cad',
  'location': {'country': 'GB'}},
 {'id': 'd90f39f0-ea5c-4583-acca-1fca527af412',
  'description': 'SUMUP *NEW LOOK BA  ON 31 DEC BCC',
  'date': '2025-01-02',
  'amount': 60.0,
  'entry_type': 'outgoing',
  'currency': 'GBP',
  'account_holder_id': '35b927b6-6fda-40aa-93b8-95b47c2b2cad',
  'location': {'country': 'GB'}},
 {'id': '3b039bbe-422e-4b9c-a305-307b4b128fe4',
  'description': 'SAINSBURYS BANK Sainsburys Bank 31DEC 21.41 ATM',
  'date': '2025-01-02',
  'amount': 50.0,
  'entry_type': 'outgoing',
  'currency': 'GBP',
  'account_holder_id': '35b927b6-6fda-40aa-93b8-95b47c2b2cad',
  'location': {'country': 'GB'}},
 {'id': '14511044-0582-4d3a-9840-e394a64ec73b',
  'description': 'KLARNA*ZOOM COMMUN  ON 26 DEC BCC',
  'date': 

## To evaluate reponses of LLM

In [6]:
df_processed.to_csv("transactions_ai_reconciled.csv")